# Smart Foodie Benchmark Test Bed

In [1]:
!rm -rf *
!curl -O https://dl.fbaipublicfiles.com/inversecooking/modelbest.ckpt
!curl -O https://raw.githubusercontent.com/tripuragorla/eranti-vijay-su21-4/master/ingr_vocab.pkl
!curl -O https://raw.githubusercontent.com/tripuragorla/eranti-vijay-su21-4/master/instr_vocab.pkl
!curl -O https://raw.githubusercontent.com/tripuragorla/eranti-vijay-su21-4/master/inversecooking.zip
!curl -o ./pizza.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Margherita_-_Five50_Aria.jpg/1083px-Margherita_-_Five50_Aria.jpg
!curl -o ./pasta.jpg https://upload.wikimedia.org/wikipedia/commons/d/dc/The_Only_Original_Alfredo_Sauce_with_Butter_and_Parmesano-Reggiano_Cheese.png
!curl -o ./burger.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/NYC-Diner-Bacon-Cheeseburger.jpg/860px-NYC-Diner-Bacon-Cheeseburger.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  396M  100  396M    0     0  26.2M      0  0:00:15  0:00:15 --:--:-- 27.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30658  100 30658    0     0   534k      0 --:--:-- --:--:-- --:--:--  534k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  453k  100  453k    0     0  4017k      0 --:--:-- --:--:-- --:--:-- 4017k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35639  100 35639    0     0   457k      0 --:--:-- --:--:-- --:--:--  457k
  % Total    % Received % Xferd  Average Speed   Tim

In [2]:
import os

!unzip -o inversecooking.zip -d "inversecooking"

for root, dirs, files in os.walk("inversecooking"):
    for file in files:
        path = root + "/" + file
        %%load {path}
        print(path)

Archive:  inversecooking.zip
  inflating: inversecooking/build_vocab.py  
  inflating: inversecooking/data_loader.py  
   creating: inversecooking/utils/
  inflating: inversecooking/utils/metrics.py  
  inflating: inversecooking/utils/ims2file.py  
  inflating: inversecooking/utils/output_utils.py  
  inflating: inversecooking/utils/tb_visualizer.py  
  inflating: inversecooking/model.py  
  inflating: inversecooking/train.py  
  inflating: inversecooking/sample.py  
  inflating: inversecooking/args.py  
   creating: inversecooking/modules/
  inflating: inversecooking/modules/multihead_attention.py  
  inflating: inversecooking/modules/encoder.py  
  inflating: inversecooking/modules/transformer_decoder.py  
  inflating: inversecooking/modules/utils.py  
inversecooking/build_vocab.py
inversecooking/train.py
inversecooking/args.py
inversecooking/sample.py
inversecooking/model.py
inversecooking/data_loader.py
inversecooking/modules/multihead_attention.py
inversecooking/modules/transforme

In [3]:
import pickle
import sys
import warnings
from random import randint

import torch
from PIL import Image
from torchvision import transforms

from inversecooking.args import get_parser
from inversecooking.model import get_model
from inversecooking.utils.output_utils import prepare_output

In [4]:
print("Initializing model...", end="")
warnings.filterwarnings("ignore")

use_gpu = True
device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
map_loc = None if torch.cuda.is_available() and use_gpu else "cpu"

ingrs_vocab = pickle.load(open("ingr_vocab.pkl", "rb"))
vocab = pickle.load(open("instr_vocab.pkl", "rb"))
ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

sys.argv = [""]
del sys
args = get_parser()
args.maxseqlen = 15
args.ingrs_only = False

model = get_model(args, ingr_vocab_size, instrs_vocab_size)
model.load_state_dict(torch.load("modelbest.ckpt", map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = False
model.recipe_only = False

transf_list_batch = [transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
to_input_transf = transforms.Compose(transf_list_batch)

greedy = [True, False, False, False]
beam = [-1, -1, -1, -1]
temperature = 1.0
numgens = 1  # len(greedy)
print("done!")

Initializing model...

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

done!


In [5]:
def predict(path):
    print("Running prediction on " + path + "...", end="")
    preds = []

    image = Image.open(path).convert("RGB")

    transf_list = [transforms.Resize(256), transforms.CenterCrop(224)]
    transform = transforms.Compose(transf_list)

    image_transf = transform(image)
    image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)

    for i in range(numgens):
        with torch.no_grad():
            outputs = model.sample(image_tensor, greedy=greedy[i],
                                   temperature=temperature, beam=beam[i], true_ingrs=None)

        ingr_ids = outputs["ingr_ids"].cpu().numpy()
        recipe_ids = outputs["recipe_ids"].cpu().numpy()
        outs, valid = prepare_output(recipe_ids[0], ingr_ids[0], ingrs_vocab, vocab)

        ingredients = list(map(lambda ingredient: {
            "ingredient": ingredient.replace("_", " ").title(),
            "carb": randint(3, 30),
            "fat": randint(6, 60),
            "cal": randint(90, 900)
        }, outs["ingrs"]))

        total = {
            "carb": sum([i["carb"] for i in ingredients]),
            "fat": sum([i["fat"] for i in ingredients]),
            "cal": sum([i["cal"] for i in ingredients])
        }

        preds.append({
            "title": outs["title"].title(),
            "ingredients": ingredients,
            "total": total,
            "recipe": outs["recipe"],
            "is_valid": valid["is_valid"]
        })

    print("done!")
    print("Predictions: " + str(preds))
    return preds

In [6]:
%%timeit
_ = predict("pizza.jpg")

Running prediction on pizza.jpg...done!
Predictions: [{'title': 'Tomato And Basil Pizza', 'ingredients': [{'ingredient': 'Cheese', 'carb': 23, 'fat': 27, 'cal': 644}, {'ingredient': 'Tomato', 'carb': 28, 'fat': 23, 'cal': 422}, {'ingredient': 'Pepper', 'carb': 18, 'fat': 39, 'cal': 98}, {'ingredient': 'Basil', 'carb': 18, 'fat': 14, 'cal': 257}, {'ingredient': 'Oil', 'carb': 4, 'fat': 20, 'cal': 664}, {'ingredient': 'Salt', 'carb': 27, 'fat': 59, 'cal': 640}, {'ingredient': 'Onion', 'carb': 27, 'fat': 7, 'cal': 674}], 'total': {'carb': 145, 'fat': 189, 'cal': 3399}, 'recipe': ['Preheat oven to 425 degrees f.', 'Spread tomato sauce on pizza crust.', 'Sprinkle with salt and pepper.', 'Top with mozzarella and basil.', 'Drizzle with olive oil.', 'Bake for 10 minutes or until cheese is melted and crust is golden brown.'], 'is_valid': True}]
Running prediction on pizza.jpg...done!
Predictions: [{'title': 'Tomato And Basil Pizza', 'ingredients': [{'ingredient': 'Cheese', 'carb': 30, 'fat': 11

In [7]:
%%timeit
_ = predict("pasta.jpg")

Running prediction on pasta.jpg...done!
Predictions: [{'title': 'Creamy Parmesan Pasta', 'ingredients': [{'ingredient': 'Cheese', 'carb': 16, 'fat': 23, 'cal': 227}, {'ingredient': 'Pepper', 'carb': 22, 'fat': 43, 'cal': 733}, {'ingredient': 'Butter', 'carb': 24, 'fat': 35, 'cal': 229}, {'ingredient': 'Milk', 'carb': 23, 'fat': 33, 'cal': 389}, {'ingredient': 'Salt', 'carb': 4, 'fat': 29, 'cal': 656}, {'ingredient': 'Pasta', 'carb': 11, 'fat': 20, 'cal': 829}], 'total': {'carb': 100, 'fat': 183, 'cal': 3063}, 'recipe': ['Cook pasta according to package directions.', 'Drain.', 'Return to pan.', 'Add butter, milk, salt and pepper.', 'Stir until butter is melted.', 'Add parmesan cheese.', 'Toss to coat.'], 'is_valid': True}]
Running prediction on pasta.jpg...done!
Predictions: [{'title': 'Creamy Parmesan Pasta', 'ingredients': [{'ingredient': 'Cheese', 'carb': 11, 'fat': 45, 'cal': 483}, {'ingredient': 'Pepper', 'carb': 3, 'fat': 19, 'cal': 830}, {'ingredient': 'Butter', 'carb': 30, 'fat'

In [8]:
%%timeit
_ = predict("burger.jpg")

Running prediction on burger.jpg...done!
Predictions: [{'title': 'Cheesy Burger', 'ingredients': [{'ingredient': 'Cheese', 'carb': 4, 'fat': 37, 'cal': 578}, {'ingredient': 'Bun', 'carb': 10, 'fat': 8, 'cal': 120}, {'ingredient': 'Pepper', 'carb': 28, 'fat': 11, 'cal': 672}, {'ingredient': 'Onion', 'carb': 5, 'fat': 44, 'cal': 700}, {'ingredient': 'Beef', 'carb': 8, 'fat': 8, 'cal': 689}], 'total': {'carb': 55, 'fat': 108, 'cal': 2759}, 'recipe': ['Heat grill to medium heat.', 'Mix meat, onions and peppers; shape into 4 (1/2-inch-thick) patties.', 'Grill patties 5 to 6 min.', 'On each side or until done (160f).', 'Top with cheese; grill 1 min.', 'Or until melted.', 'Fill buns with cheeseburgers and remaining ingredients.'], 'is_valid': True}]
Running prediction on burger.jpg...done!
Predictions: [{'title': 'Cheesy Burger', 'ingredients': [{'ingredient': 'Cheese', 'carb': 9, 'fat': 17, 'cal': 379}, {'ingredient': 'Bun', 'carb': 26, 'fat': 19, 'cal': 698}, {'ingredient': 'Pepper', 'carb'